In [1]:
from SBM_SDE import *
from obs_and_flow_classes_and_functions import *
import torch
from torch import nn
import torch.distributions as d
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [2]:
torch.manual_seed(0)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .2 #SDE discretization timestep.
t = 150 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 1e-3
niter = 5000
piter = 100
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.

In [3]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [4]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [5]:
def calc_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_GEN, TEMP_REF)
    euler_maruyama_state_sample_object = d.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [6]:
def train(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, SDEFLOW, ObsModel, csv_to_obs_df, DATA_CSV, OBS_ERROR_SCALE, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT):
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM + 1, T, OBS_ERROR_SCALE) 
    obs_model = ObsModel(DEVICE, obs_times, DT, obs_means[:-1, :], obs_error[:, :-1]) #Hack for bypassing ObsModel and SDEFlow dimension mismatch issue.
    net = SDEFlow(DEVICE, BATCH_SIZE, obs_model, STATE_DIM, T, DT, N).to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr = L_R) 
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), PARAMS_DICT) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Assign initial conditions to C_PATH.
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for iter in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net() #Obtain paths with solutions at times after t0.
            C_PATH = torch.cat([C0, C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            if iter <= PRETRAIN_ITER:
                l1_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l1_norm.item())
                #l2_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                #l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                #best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                #l2_norm.backward()
                #norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
                l1_norm.backward()
            else:
                log_lik = calc_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), DT, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF)
                neg_ELBO = -log_lik.mean() - obs_model(C_PATH) + log_prob.mean()
                best_loss_ELBO = neg_ELBO if neg_ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(neg_ELBO.item())
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\n C_PATH =', C_PATH)
                neg_ELBO.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if iter % 100000 == 0 and iter > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()

In [ ]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON)


Train Diffusion:   0%|          | 0/5000 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000006813.896093. Best norm loss value is: 68138.9609375.

C_PATH mean = tensor([[0.9574, 0.6532, 0.6704],
        [0.9427, 0.6480, 0.6632]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 0.6694,  0.5031,  0.6758],
         [ 0.9921,  0.9276,  0.5271],
         ...,
         [ 0.5774,  0.6779,  0.7837],
         [ 0.3762,  0.5808,  0.4681],
         [ 0.6598,  0.7157,  0.3054]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.6306,  0.8222,  0.6741],
         [ 1.0157,  0.4546,  0.9029],
         ...,
         [ 0.6804,  0.6085,  0.7170],
         [ 0.7476,  0.6574,  0.5636],
         [ 1.6166,  0.6980,  0.3771]]], grad_fn=<CatBackward>)



Train Diffusion:   0%|          | 10/5000 [00:06<57:47,  1.44it/s] 

Moving average norm loss at 10 iterations is: 64226.754296875. Best norm loss value is: 61744.9375.

C_PATH mean = tensor([[4.9908, 0.1272, 0.3737],
        [4.8272, 0.1318, 0.3741]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 0.7348,  0.6258,  0.2338],
         [ 1.8357,  0.4334,  0.2583],
         ...,
         [ 6.3464,  0.2027,  0.4205],
         [ 5.2758,  0.2083,  0.3851],
         [ 0.8174,  0.2376,  0.2581]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.4793,  4.4348,  0.2016],
         [ 0.4264,  0.2306,  0.2186],
         ...,
         [ 1.9857,  0.2264,  0.5069],
         [ 1.3734,  0.2498,  0.4501],
         [ 7.0078,  0.2384,  0.2231]]], grad_fn=<CatBackward>)



Train Diffusion:   0%|          | 20/5000 [00:13<55:10,  1.50it/s]

...


Train Diffusion:  59%|█████▉    | 2950/5000 [31:59<20:39,  1.65it/s]

Moving average ELBO loss at 2950 iterations is: -9127.90302734375. Best ELBO loss value is: -9611.2919921875.

C_PATH mean = tensor([[41.1816,  0.1511,  0.7212],
        [41.2194,  0.1482,  0.7350]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [4.5186e+01, 4.4606e-02, 5.6520e-01],
         [4.4843e+01, 4.7100e-02, 4.7199e-01],
         ...,
         [4.2219e+01, 1.7291e-01, 6.1332e-01],
         [4.1630e+01, 1.3577e-01, 6.0369e-01],
         [4.2160e+01, 8.4412e-02, 7.5987e-01]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [4.6167e+01, 1.0802e-01, 1.6933e-01],
         [4.6137e+01, 1.5169e-01, 3.1309e-01],
         ...,
         [4.0746e+01, 9.0317e-02, 8.6429e-01],
         [4.1456e+01, 7.0329e-02, 8.5481e-01],
         [4.1112e+01, 8.4144e-02, 7.3500e-01]]], grad_fn=<CatBackward>)



Train Diffusion:  59%|█████▉    | 2960/5000 [32:05<18:48,  1.81it/s]

Moving average ELBO loss at 2960 iterations is: -9369.697265625. Best ELBO loss value is: -9650.9072265625.

C_PATH mean = tensor([[41.2410,  0.1560,  0.7220],
        [41.2223,  0.1438,  0.6955]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [4.6161e+01, 1.0995e-01, 1.5551e-01],
         [4.6226e+01, 1.5110e-01, 3.1419e-01],
         ...,
         [4.1899e+01, 9.4973e-02, 8.0366e-01],
         [4.1472e+01, 7.0290e-02, 8.3470e-01],
         [4.1776e+01, 6.3588e-02, 7.1866e-01]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [4.5212e+01, 1.7788e-02, 6.1937e-01],
         [4.5063e+01, 2.8108e-02, 4.5628e-01],
         ...,
         [4.0906e+01, 1.5585e-01, 5.8265e-01],
         [4.1427e+01, 1.3003e-01, 5.5868e-01],
         [4.0933e+01, 9.6720e-02, 7.2737e-01]]], grad_fn=<CatBackward>)



Train Diffusion:  59%|█████▉    | 2962/5000 [32:06<18:40,  1.82it/s]

In [ ]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_SS, SCON_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON)

In [ ]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_C, SAWB_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB)

In [ ]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_SS, SAWB_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB)

In [ ]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_C, SAWB_ECA_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA)

In [ ]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_SS, SAWB_ECA_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA)